In [1]:
# base
import pandas as pd
import numpy as np
from tqdm import tqdm
import lime
import math
import imblearn
import seaborn as sns
import shap
import copy
from matplotlib import pyplot as plt
import pycaret
from pycaret.classification import *
import pickle
from sklearn.model_selection import train_test_split
import time
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
import statistics

# warning
import warnings
warnings.filterwarnings("ignore")

In [2]:
selected_var_except_del=pd.read_csv("C:\\Users\\srija\\OneDrive\\Desktop\\MENTORSHIP\\amex kaggle competition\\selected var except del.csv")
selected_var_except_del

,selected except del
0,S_3
1,S_23
2,S_25
3,S_8
4,B_9
5,B_2
6,B_18
7,B_7
8,B_5
9,B_40


In [3]:
test_full = pd.read_feather('D:\\AMEX\\test_data.ftr')
test_full

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631348,0.001912,0.010727,0.814453,0.007545,0.168701,0.009972,0.002348,...,NaN,NaN,NaN,NaN,0.004669,NaN,NaN,NaN,0.008278,NaN
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.586914,0.005276,0.011024,0.811035,0.001817,0.241333,0.000166,0.009132,...,NaN,NaN,NaN,0.000142,0.004940,0.009018,NaN,0.003695,0.003754,0.001460
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.608887,0.003326,0.016388,1.004883,0.000114,0.267090,0.004196,0.004192,...,NaN,NaN,NaN,0.000074,0.002113,0.004658,NaN,0.003155,0.002155,0.006481
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-05-20,0.614746,0.009064,0.021667,0.816406,0.009720,0.188965,0.004124,0.015327,...,NaN,NaN,NaN,0.004742,0.006393,0.002890,NaN,0.006042,0.005207,0.007858
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-06-15,0.591797,0.238770,0.015930,0.810547,0.002026,0.180054,0.000731,0.011284,...,NaN,NaN,NaN,0.008133,0.004330,0.008385,NaN,0.001008,0.007420,0.009468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11363757,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2018-12-19,0.440918,0.000144,0.008636,0.818359,0.001797,NaN,0.006420,0.002710,...,NaN,NaN,NaN,0.006512,0.008873,0.001236,NaN,0.009659,0.005520,0.001351
11363758,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2019-01-19,0.445557,0.006832,0.009636,0.815430,0.009872,NaN,0.003611,0.006920,...,NaN,NaN,NaN,0.003374,0.009003,0.004181,NaN,0.007835,0.000256,0.004917
11363759,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2019-02-17,0.459961,0.005245,0.004421,0.818848,0.000665,NaN,0.000559,0.007057,...,NaN,NaN,NaN,0.007797,0.007740,0.008194,NaN,0.009384,0.003593,0.005604
11363760,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2019-03-19,0.460449,0.002663,0.009529,0.816406,0.009026,NaN,0.008530,0.006851,...,NaN,NaN,NaN,0.007195,0.008453,0.004776,NaN,0.006199,0.005604,0.005970


In [4]:
cat_feat_full = ['D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
delinquent_feat = [i for i in test_full.columns if i.startswith('D_') and i not in cat_feat_full]
cols_to_read=['customer_ID','S_2']+list(selected_var_except_del['selected except del'])+delinquent_feat
cols_to_read

['customer_ID',
 'S_2',
 'S_3',
 'S_23',
 'S_25',
 'S_8',
 'B_9',
 'B_2',
 'B_18',
 'B_7',
 'B_5',
 'B_40',
 'B_33',
 'B_3',
 'B_8',
 'R_1',
 'R_10',
 'R_3',
 'R_2',
 'R_6',
 'R_27',
 'R_16',
 'R_7',
 'R_5',
 'P_2',
 'B_30',
 'D_120',
 'D_39',
 'D_41',
 'D_42',
 'D_43',
 'D_44',
 'D_45',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'D_50',
 'D_51',
 'D_52',
 'D_53',
 'D_54',
 'D_55',
 'D_56',
 'D_58',
 'D_59',
 'D_60',
 'D_61',
 'D_62',
 'D_65',
 'D_69',
 'D_70',
 'D_71',
 'D_72',
 'D_73',
 'D_74',
 'D_75',
 'D_76',
 'D_77',
 'D_78',
 'D_79',
 'D_80',
 'D_81',
 'D_82',
 'D_83',
 'D_84',
 'D_86',
 'D_87',
 'D_88',
 'D_89',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'D_96',
 'D_102',
 'D_103',
 'D_104',
 'D_105',
 'D_106',
 'D_107',
 'D_108',
 'D_109',
 'D_110',
 'D_111',
 'D_112',
 'D_113',
 'D_115',
 'D_118',
 'D_119',
 'D_121',
 'D_122',
 'D_123',
 'D_124',
 'D_125',
 'D_127',
 'D_128',
 'D_129',
 'D_130',
 'D_131',
 'D_132',
 'D_133',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_139',
 'D

In [5]:
test = pd.read_feather('D:\\AMEX\\test_data.ftr',columns=cols_to_read)
test

,customer_ID,S_2,S_3,S_23,S_25,S_8,B_9,B_2,B_18,B_7,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.168701,0.134888,0.971191,0.321777,0.001013,0.814453,0.592285,1.250977,...,NaN,NaN,NaN,NaN,0.004669,NaN,NaN,NaN,0.008278,NaN
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.241333,0.138550,0.975098,0.170776,0.005535,0.811035,0.590820,0.234131,...,NaN,NaN,NaN,0.000142,0.004940,0.009018,NaN,0.003695,0.003754,0.001460
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.267090,0.136475,0.978516,0.172729,0.000023,1.004883,0.591309,0.126953,...,NaN,NaN,NaN,0.000074,0.002113,0.004658,NaN,0.003155,0.002155,0.006481
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-05-20,0.188965,0.131470,0.976562,0.171143,0.007206,0.816406,0.590820,0.144653,...,NaN,NaN,NaN,0.004742,0.006393,0.002890,NaN,0.006042,0.005207,0.007858
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-06-15,0.180054,0.139648,0.975586,0.320068,0.000569,0.810547,0.593262,0.178223,...,NaN,NaN,NaN,0.008133,0.004330,0.008385,NaN,0.001008,0.007420,0.009468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11363757,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2018-12-19,NaN,0.135986,0.977051,0.009285,0.009430,0.818359,0.694824,0.126709,...,NaN,NaN,NaN,0.006512,0.008873,0.001236,NaN,0.009659,0.005520,0.001351
11363758,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2019-01-19,NaN,0.140137,0.970703,0.006702,0.004105,0.815430,0.690430,0.171509,...,NaN,NaN,NaN,0.003374,0.009003,0.004181,NaN,0.007835,0.000256,0.004917
11363759,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2019-02-17,NaN,0.138550,0.969238,0.005974,0.006485,0.818848,0.696777,0.133789,...,NaN,NaN,NaN,0.007797,0.007740,0.008194,NaN,0.009384,0.003593,0.005604
11363760,fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...,2019-03-19,NaN,0.135620,0.978027,0.007729,0.009315,0.816406,0.689453,0.130005,...,NaN,NaN,NaN,0.007195,0.008453,0.004776,NaN,0.006199,0.005604,0.005970


In [6]:
cat_feat = ['B_30', 'D_120']
drop_feat = ['customer_ID', 'S_2']
num_feat = sorted([f for f in test.columns if f not in cat_feat + drop_feat])
spend_feat      = [i for i in num_feat if i.startswith('S_')]
payment_feat    = [i for i in num_feat if i.startswith('P_')]
balance_feat    = [i for i in num_feat if i.startswith('B_')]
risk_feat       = [i for i in num_feat if i.startswith('R_')]

In [7]:
features=[f for f in test.columns if f not in drop_feat + ['target']]

In [8]:
num_agg1 = test.groupby("customer_ID")[num_feat].agg(['mean', 'std', 'min', 'max'])
num_agg1.columns = ['_'.join(x) for x in num_agg1.columns]
num_agg1.reset_index(level=0, inplace=True)
num_agg1

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,S_25_min,S_25_max,S_3_mean,S_3_std,S_3_min,S_3_max,S_8_mean,S_8_std,S_8_min,S_8_max
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590820,0.002158,0.585449,0.593262,0.898438,0.100392,0.810547,1.009766,0.009735,...,0.969727,0.978516,0.188599,0.039416,0.149414,0.267090,0.271240,0.103693,0.170776,0.464111
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,0.002412,1.000977,1.008789,1.003906,0.002626,1.000977,1.008789,0.006161,...,0.969238,0.977539,0.117249,0.043798,0.055817,0.172974,0.840332,0.110109,0.604980,1.004883
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748535,0.210965,0.584473,1.009766,0.901367,0.100461,0.810059,1.009766,0.008553,...,0.969727,0.978027,0.161499,0.015294,0.132568,0.176636,0.336182,0.204347,0.007782,0.759766
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.145020,0.008443,0.136230,0.165649,0.090881,0.045857,0.032684,0.205688,0.666992,...,0.969238,0.978027,0.294434,0.063766,0.181152,0.382324,0.553711,0.114457,0.466064,0.752441
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210327,0.002576,0.205566,0.213257,0.041931,0.024126,0.020172,0.106750,0.616699,...,0.161621,0.978027,0.165161,0.005721,0.155518,0.174927,0.281738,0.236115,0.005058,0.605469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.846680,0.175508,0.647949,1.006836,0.815430,0.002537,0.811035,0.819336,0.024124,...,0.969727,0.978027,0.125366,0.020369,0.102295,0.162964,0.362549,0.002966,0.357910,0.367188
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.436768,0.239812,0.186279,1.005859,0.411621,0.355722,0.082520,1.009766,0.037567,...,0.969238,0.978516,0.833008,0.241404,0.400635,1.226562,0.195068,0.124905,0.003311,0.407959
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.531738,0.140522,0.151123,0.592285,0.902344,0.241377,0.171143,1.009766,0.028580,...,0.291260,0.979004,0.242065,0.084389,0.158691,0.380859,0.408936,0.139682,0.168335,0.665039
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.018127,0.020977,0.009804,0.087463,0.134888,0.025084,0.103333,0.192871,0.119263,...,0.969727,0.978516,0.202881,0.014041,0.179688,0.230835,0.098633,0.145576,0.001014,0.332031


In [9]:
num_feat_withid=['customer_ID']
for i in num_feat:
    num_feat_withid.append(i)
num_feat_withid 

['customer_ID',
 'B_18',
 'B_2',
 'B_3',
 'B_33',
 'B_40',
 'B_5',
 'B_7',
 'B_8',
 'B_9',
 'D_102',
 'D_103',
 'D_104',
 'D_105',
 'D_106',
 'D_107',
 'D_108',
 'D_109',
 'D_110',
 'D_111',
 'D_112',
 'D_113',
 'D_115',
 'D_118',
 'D_119',
 'D_121',
 'D_122',
 'D_123',
 'D_124',
 'D_125',
 'D_127',
 'D_128',
 'D_129',
 'D_130',
 'D_131',
 'D_132',
 'D_133',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_139',
 'D_140',
 'D_141',
 'D_142',
 'D_143',
 'D_144',
 'D_145',
 'D_39',
 'D_41',
 'D_42',
 'D_43',
 'D_44',
 'D_45',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'D_50',
 'D_51',
 'D_52',
 'D_53',
 'D_54',
 'D_55',
 'D_56',
 'D_58',
 'D_59',
 'D_60',
 'D_61',
 'D_62',
 'D_65',
 'D_69',
 'D_70',
 'D_71',
 'D_72',
 'D_73',
 'D_74',
 'D_75',
 'D_76',
 'D_77',
 'D_78',
 'D_79',
 'D_80',
 'D_81',
 'D_82',
 'D_83',
 'D_84',
 'D_86',
 'D_87',
 'D_88',
 'D_89',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'D_96',
 'P_2',
 'R_1',
 'R_10',
 'R_16',
 'R_2',
 'R_27',
 'R_3',
 'R_5',
 'R_6',
 'R_7',
 'S_

In [10]:
col_name2=['customer_ID']
for i in num_feat:
    col_name2.append(i+str('_last obs'))
col_name2

['customer_ID',
 'B_18_last obs',
 'B_2_last obs',
 'B_3_last obs',
 'B_33_last obs',
 'B_40_last obs',
 'B_5_last obs',
 'B_7_last obs',
 'B_8_last obs',
 'B_9_last obs',
 'D_102_last obs',
 'D_103_last obs',
 'D_104_last obs',
 'D_105_last obs',
 'D_106_last obs',
 'D_107_last obs',
 'D_108_last obs',
 'D_109_last obs',
 'D_110_last obs',
 'D_111_last obs',
 'D_112_last obs',
 'D_113_last obs',
 'D_115_last obs',
 'D_118_last obs',
 'D_119_last obs',
 'D_121_last obs',
 'D_122_last obs',
 'D_123_last obs',
 'D_124_last obs',
 'D_125_last obs',
 'D_127_last obs',
 'D_128_last obs',
 'D_129_last obs',
 'D_130_last obs',
 'D_131_last obs',
 'D_132_last obs',
 'D_133_last obs',
 'D_134_last obs',
 'D_135_last obs',
 'D_136_last obs',
 'D_137_last obs',
 'D_138_last obs',
 'D_139_last obs',
 'D_140_last obs',
 'D_141_last obs',
 'D_142_last obs',
 'D_143_last obs',
 'D_144_last obs',
 'D_145_last obs',
 'D_39_last obs',
 'D_41_last obs',
 'D_42_last obs',
 'D_43_last obs',
 'D_44_last obs

In [11]:
dev_last=test.groupby("customer_ID").tail(1)[num_feat_withid]
dev_last.columns=col_name2
dev_last.fillna(value=0,inplace=True)
dev_last

,customer_ID,B_18_last obs,B_2_last obs,B_3_last obs,B_33_last obs,B_40_last obs,B_5_last obs,B_7_last obs,B_8_last obs,B_9_last obs,...,R_2_last obs,R_27_last obs,R_3_last obs,R_5_last obs,R_6_last obs,R_7_last obs,S_23_last obs,S_25_last obs,S_3_last obs,S_8_last obs
8,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590332,1.009766,0.003576,1.007812,0.375488,0.050201,0.163452,1.008789,0.005264,...,0.007511,1.001953,0.109436,0.005444,0.002590,0.005943,0.135742,0.970703,0.149414,0.464111
21,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,1.008789,0.011383,1.004883,0.005497,0.135864,0.014465,1.008789,0.017456,...,0.002270,1.000977,0.008148,0.004131,0.009712,0.006783,0.135620,0.974609,0.112183,0.768555
34,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.585938,0.810059,0.015945,1.000000,0.096802,0.004852,0.090210,1.009766,0.008629,...,0.000715,1.003906,0.001311,0.006618,0.007610,0.005554,0.133057,0.970215,0.166138,0.122986
47,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.149780,0.205688,0.498535,0.001155,0.228394,0.022949,0.206177,0.009773,0.185303,...,0.009117,1.001953,0.104187,0.002604,0.008202,0.009407,0.137939,0.978027,0.181152,0.479004
60,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210571,0.038025,0.831055,0.001682,0.081055,0.011124,0.261719,1.005859,0.656738,...,0.007126,0.011078,0.004543,0.004383,0.001731,0.000699,0.188965,0.339600,0.168335,0.244995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11363714,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.688965,0.816895,0.028900,1.007812,0.190796,0.008644,0.102905,0.000648,0.003752,...,0.006279,1.001953,0.006641,0.003819,0.005413,0.006069,0.140137,0.971191,0.162964,0.364502
11363727,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.186279,0.082520,0.070496,0.007069,0.366943,0.031952,0.268066,1.003906,0.213379,...,0.009575,0.015038,0.508301,0.008873,0.004917,0.007675,0.136353,0.976562,0.856934,0.315674
11363740,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.590332,1.003906,0.014290,1.006836,0.225586,0.026245,0.173950,1.007812,0.007355,...,0.003794,-0.000828,0.408447,0.001188,0.009895,0.007656,0.137817,0.978027,0.321045,0.601074
11363753,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.087463,0.192871,0.099915,0.008469,0.408203,0.009506,0.541504,1.004883,0.019562,...,0.001982,1.000000,0.005043,0.008186,0.001944,0.007107,0.132690,0.978516,0.230835,0.009270


In [12]:
col_name=['customer_ID']
for i in num_feat:
    col_name.append(i+str('_last3_avg'))
col_name    

['customer_ID',
 'B_18_last3_avg',
 'B_2_last3_avg',
 'B_3_last3_avg',
 'B_33_last3_avg',
 'B_40_last3_avg',
 'B_5_last3_avg',
 'B_7_last3_avg',
 'B_8_last3_avg',
 'B_9_last3_avg',
 'D_102_last3_avg',
 'D_103_last3_avg',
 'D_104_last3_avg',
 'D_105_last3_avg',
 'D_106_last3_avg',
 'D_107_last3_avg',
 'D_108_last3_avg',
 'D_109_last3_avg',
 'D_110_last3_avg',
 'D_111_last3_avg',
 'D_112_last3_avg',
 'D_113_last3_avg',
 'D_115_last3_avg',
 'D_118_last3_avg',
 'D_119_last3_avg',
 'D_121_last3_avg',
 'D_122_last3_avg',
 'D_123_last3_avg',
 'D_124_last3_avg',
 'D_125_last3_avg',
 'D_127_last3_avg',
 'D_128_last3_avg',
 'D_129_last3_avg',
 'D_130_last3_avg',
 'D_131_last3_avg',
 'D_132_last3_avg',
 'D_133_last3_avg',
 'D_134_last3_avg',
 'D_135_last3_avg',
 'D_136_last3_avg',
 'D_137_last3_avg',
 'D_138_last3_avg',
 'D_139_last3_avg',
 'D_140_last3_avg',
 'D_141_last3_avg',
 'D_142_last3_avg',
 'D_143_last3_avg',
 'D_144_last3_avg',
 'D_145_last3_avg',
 'D_39_last3_avg',
 'D_41_last3_avg',
 

In [13]:
dev_last3_avg = test.groupby("customer_ID").tail(3).groupby("customer_ID")[num_feat].agg(['mean'])
dev_last3_avg.reset_index(level=0, inplace=True)
dev_last3_avg.columns=col_name
dev_last3_avg.fillna(value=0,inplace=True)
dev_last3_avg

,customer_ID,B_18_last3_avg,B_2_last3_avg,B_3_last3_avg,B_33_last3_avg,B_40_last3_avg,B_5_last3_avg,B_7_last3_avg,B_8_last3_avg,B_9_last3_avg,...,R_2_last3_avg,R_27_last3_avg,R_3_last3_avg,R_5_last3_avg,R_6_last3_avg,R_7_last3_avg,S_23_last3_avg,S_25_last3_avg,S_3_last3_avg,S_8_last3_avg
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590332,0.942383,0.011848,1.005859,0.377441,0.036346,0.160156,1.007812,0.005058,...,0.003548,1.005859,0.107422,0.003557,0.003626,0.003531,0.134766,0.972656,0.159302,0.369141
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.003906,1.004883,0.006832,1.004883,0.016907,0.067444,0.022202,1.005859,0.055328,...,0.004589,1.004883,0.005573,0.003674,0.004379,0.005783,0.134399,0.974609,0.093994,0.828613
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.589355,0.810547,0.013855,1.004883,0.104980,0.005909,0.101807,1.004883,0.007626,...,0.003283,1.005859,0.038116,0.004925,0.006721,0.005119,0.136719,0.970703,0.171021,0.191406
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.153076,0.132568,0.596191,0.001772,0.251709,0.027100,0.273193,0.009483,0.352783,...,0.007641,1.001953,0.172974,0.004025,0.008682,0.008484,0.136230,0.975098,0.222046,0.481934
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.209839,0.032318,0.806641,0.003056,0.080200,0.012115,0.283691,1.003906,0.642578,...,0.003429,0.012138,0.003700,0.005741,0.001112,0.004364,0.257080,0.340332,0.166260,0.323730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.677734,0.815430,0.024567,1.006836,0.288086,0.005611,0.114441,0.004074,0.003714,...,0.007408,1.003906,0.003300,0.004162,0.004726,0.005219,0.137085,0.972168,0.156250,0.361572
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.312256,0.138062,0.061432,0.003267,0.225952,0.017090,0.174683,1.002930,0.200562,...,0.007435,0.014755,0.406006,0.005356,0.005482,0.005836,0.138306,0.974609,0.844238,0.165283
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.589355,1.006836,0.009552,1.003906,0.245117,0.024353,0.186157,1.006836,0.005177,...,0.004749,0.002638,0.437500,0.003611,0.007565,0.004459,0.134277,0.975098,0.352295,0.462402
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.037598,0.158691,0.111084,0.007320,0.394531,0.011360,0.520996,1.004883,0.019592,...,0.004295,1.000000,0.006256,0.006416,0.003050,0.004829,0.134155,0.975586,0.208008,0.008316


In [14]:
num_agg2=pd.merge(num_agg1,dev_last, left_on = 'customer_ID', right_on = 'customer_ID',how = 'outer')
num_agg2

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_last obs,R_27_last obs,R_3_last obs,R_5_last obs,R_6_last obs,R_7_last obs,S_23_last obs,S_25_last obs,S_3_last obs,S_8_last obs
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590820,0.002158,0.585449,0.593262,0.898438,0.100392,0.810547,1.009766,0.009735,...,0.007511,1.001953,0.109436,0.005444,0.002590,0.005943,0.135742,0.970703,0.149414,0.464111
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,0.002412,1.000977,1.008789,1.003906,0.002626,1.000977,1.008789,0.006161,...,0.002270,1.000977,0.008148,0.004131,0.009712,0.006783,0.135620,0.974609,0.112183,0.768555
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748535,0.210965,0.584473,1.009766,0.901367,0.100461,0.810059,1.009766,0.008553,...,0.000715,1.003906,0.001311,0.006618,0.007610,0.005554,0.133057,0.970215,0.166138,0.122986
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.145020,0.008443,0.136230,0.165649,0.090881,0.045857,0.032684,0.205688,0.666992,...,0.009117,1.001953,0.104187,0.002604,0.008202,0.009407,0.137939,0.978027,0.181152,0.479004
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210327,0.002576,0.205566,0.213257,0.041931,0.024126,0.020172,0.106750,0.616699,...,0.007126,0.011078,0.004543,0.004383,0.001731,0.000699,0.188965,0.339600,0.168335,0.244995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.846680,0.175508,0.647949,1.006836,0.815430,0.002537,0.811035,0.819336,0.024124,...,0.006279,1.001953,0.006641,0.003819,0.005413,0.006069,0.140137,0.971191,0.162964,0.364502
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.436768,0.239812,0.186279,1.005859,0.411621,0.355722,0.082520,1.009766,0.037567,...,0.009575,0.015038,0.508301,0.008873,0.004917,0.007675,0.136353,0.976562,0.856934,0.315674
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.531738,0.140522,0.151123,0.592285,0.902344,0.241377,0.171143,1.009766,0.028580,...,0.003794,-0.000828,0.408447,0.001188,0.009895,0.007656,0.137817,0.978027,0.321045,0.601074
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.018127,0.020977,0.009804,0.087463,0.134888,0.025084,0.103333,0.192871,0.119263,...,0.001982,1.000000,0.005043,0.008186,0.001944,0.007107,0.132690,0.978516,0.230835,0.009270


In [15]:
for i in num_feat:
    last=i+str('_last obs')
    mean=i+str('_mean')
    std=i+str('_std')
    flag_last=i+str('_flag_last')
    num_agg2[flag_last]=num_agg2[last]>num_agg2[mean]+3*num_agg2[std] 
    num_agg2[flag_last]=num_agg2[flag_last].astype(int)
num_agg2    

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_flag_last,R_27_flag_last,R_3_flag_last,R_5_flag_last,R_6_flag_last,R_7_flag_last,S_23_flag_last,S_25_flag_last,S_3_flag_last,S_8_flag_last
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590820,0.002158,0.585449,0.593262,0.898438,0.100392,0.810547,1.009766,0.009735,...,0,0,0,0,0,0,0,0,0,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,0.002412,1.000977,1.008789,1.003906,0.002626,1.000977,1.008789,0.006161,...,0,0,0,0,0,0,0,0,0,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748535,0.210965,0.584473,1.009766,0.901367,0.100461,0.810059,1.009766,0.008553,...,0,0,0,0,0,0,0,0,0,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.145020,0.008443,0.136230,0.165649,0.090881,0.045857,0.032684,0.205688,0.666992,...,0,0,0,0,0,0,0,0,0,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210327,0.002576,0.205566,0.213257,0.041931,0.024126,0.020172,0.106750,0.616699,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.846680,0.175508,0.647949,1.006836,0.815430,0.002537,0.811035,0.819336,0.024124,...,0,0,0,0,0,0,0,0,0,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.436768,0.239812,0.186279,1.005859,0.411621,0.355722,0.082520,1.009766,0.037567,...,0,0,0,0,0,0,0,0,0,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.531738,0.140522,0.151123,0.592285,0.902344,0.241377,0.171143,1.009766,0.028580,...,0,0,0,0,0,0,0,0,0,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.018127,0.020977,0.009804,0.087463,0.134888,0.025084,0.103333,0.192871,0.119263,...,0,0,0,0,0,0,0,0,0,0


In [16]:
num_agg3=pd.merge(num_agg2,dev_last3_avg, left_on = 'customer_ID', right_on = 'customer_ID',how = 'outer')
num_agg3

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_last3_avg,R_27_last3_avg,R_3_last3_avg,R_5_last3_avg,R_6_last3_avg,R_7_last3_avg,S_23_last3_avg,S_25_last3_avg,S_3_last3_avg,S_8_last3_avg
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590820,0.002158,0.585449,0.593262,0.898438,0.100392,0.810547,1.009766,0.009735,...,0.003548,1.005859,0.107422,0.003557,0.003626,0.003531,0.134766,0.972656,0.159302,0.369141
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,0.002412,1.000977,1.008789,1.003906,0.002626,1.000977,1.008789,0.006161,...,0.004589,1.004883,0.005573,0.003674,0.004379,0.005783,0.134399,0.974609,0.093994,0.828613
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748535,0.210965,0.584473,1.009766,0.901367,0.100461,0.810059,1.009766,0.008553,...,0.003283,1.005859,0.038116,0.004925,0.006721,0.005119,0.136719,0.970703,0.171021,0.191406
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.145020,0.008443,0.136230,0.165649,0.090881,0.045857,0.032684,0.205688,0.666992,...,0.007641,1.001953,0.172974,0.004025,0.008682,0.008484,0.136230,0.975098,0.222046,0.481934
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210327,0.002576,0.205566,0.213257,0.041931,0.024126,0.020172,0.106750,0.616699,...,0.003429,0.012138,0.003700,0.005741,0.001112,0.004364,0.257080,0.340332,0.166260,0.323730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.846680,0.175508,0.647949,1.006836,0.815430,0.002537,0.811035,0.819336,0.024124,...,0.007408,1.003906,0.003300,0.004162,0.004726,0.005219,0.137085,0.972168,0.156250,0.361572
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.436768,0.239812,0.186279,1.005859,0.411621,0.355722,0.082520,1.009766,0.037567,...,0.007435,0.014755,0.406006,0.005356,0.005482,0.005836,0.138306,0.974609,0.844238,0.165283
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.531738,0.140522,0.151123,0.592285,0.902344,0.241377,0.171143,1.009766,0.028580,...,0.004749,0.002638,0.437500,0.003611,0.007565,0.004459,0.134277,0.975098,0.352295,0.462402
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.018127,0.020977,0.009804,0.087463,0.134888,0.025084,0.103333,0.192871,0.119263,...,0.004295,1.000000,0.006256,0.006416,0.003050,0.004829,0.134155,0.975586,0.208008,0.008316


In [17]:
for i in num_feat:
    last3=i+str('_last3_avg')
    mean=i+str('_mean')
    std=i+str('_std')
    flag_last3=i+str('_flag_last3')
    num_agg3[flag_last3]=num_agg3[last3]>num_agg3[mean]+3*num_agg3[std] 
    num_agg3[flag_last3]=num_agg3[flag_last3].astype(int)
num_agg3    

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_flag_last3,R_27_flag_last3,R_3_flag_last3,R_5_flag_last3,R_6_flag_last3,R_7_flag_last3,S_23_flag_last3,S_25_flag_last3,S_3_flag_last3,S_8_flag_last3
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590820,0.002158,0.585449,0.593262,0.898438,0.100392,0.810547,1.009766,0.009735,...,0,0,0,0,0,0,0,0,0,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,0.002412,1.000977,1.008789,1.003906,0.002626,1.000977,1.008789,0.006161,...,0,0,0,0,0,0,0,0,0,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748535,0.210965,0.584473,1.009766,0.901367,0.100461,0.810059,1.009766,0.008553,...,0,0,0,0,0,0,0,0,0,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.145020,0.008443,0.136230,0.165649,0.090881,0.045857,0.032684,0.205688,0.666992,...,0,0,0,0,0,0,0,0,0,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210327,0.002576,0.205566,0.213257,0.041931,0.024126,0.020172,0.106750,0.616699,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.846680,0.175508,0.647949,1.006836,0.815430,0.002537,0.811035,0.819336,0.024124,...,0,0,0,0,0,0,0,0,0,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.436768,0.239812,0.186279,1.005859,0.411621,0.355722,0.082520,1.009766,0.037567,...,0,0,0,0,0,0,0,0,0,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.531738,0.140522,0.151123,0.592285,0.902344,0.241377,0.171143,1.009766,0.028580,...,0,0,0,0,0,0,0,0,0,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.018127,0.020977,0.009804,0.087463,0.134888,0.025084,0.103333,0.192871,0.119263,...,0,0,0,0,0,0,0,0,0,0


In [19]:
cat_agg = test.groupby("customer_ID")[cat_feat].agg(['count', 'last', 'nunique'])
cat_agg.columns = ['_'.join(x) for x in cat_agg.columns]
cat_agg=pd.get_dummies(cat_agg,columns=list(cat_agg.columns))  
cat_agg.reset_index(level=0, inplace=True)
cat_agg

,customer_ID,B_30_count_0,B_30_count_1,B_30_count_2,B_30_count_3,B_30_count_4,B_30_count_5,B_30_count_6,B_30_count_7,B_30_count_8,...,D_120_count_9,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1


In [20]:
final_agg=pd.merge(num_agg3,cat_agg, left_on = 'customer_ID', right_on = 'customer_ID',how = 'outer')
final_agg

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,D_120_count_9,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590820,0.002158,0.585449,0.593262,0.898438,0.100392,0.810547,1.009766,0.009735,...,0,0,0,0,0,0,1,0,1,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,0.002412,1.000977,1.008789,1.003906,0.002626,1.000977,1.008789,0.006161,...,0,0,0,0,1,1,0,0,1,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748535,0.210965,0.584473,1.009766,0.901367,0.100461,0.810059,1.009766,0.008553,...,0,0,0,0,1,1,0,0,1,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.145020,0.008443,0.136230,0.165649,0.090881,0.045857,0.032684,0.205688,0.666992,...,0,0,0,0,1,1,0,0,0,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210327,0.002576,0.205566,0.213257,0.041931,0.024126,0.020172,0.106750,0.616699,...,0,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.846680,0.175508,0.647949,1.006836,0.815430,0.002537,0.811035,0.819336,0.024124,...,0,0,0,0,1,1,0,0,1,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.436768,0.239812,0.186279,1.005859,0.411621,0.355722,0.082520,1.009766,0.037567,...,0,0,0,0,1,1,0,0,0,1
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.531738,0.140522,0.151123,0.592285,0.902344,0.241377,0.171143,1.009766,0.028580,...,0,0,0,0,1,1,0,0,0,1
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.018127,0.020977,0.009804,0.087463,0.134888,0.025084,0.103333,0.192871,0.119263,...,0,0,0,0,1,0,1,0,0,1


In [21]:
#dropping the std features
for i in num_feat:
    std=i+str('_std')
    final_agg.drop(columns=std,inplace=True)
final_agg

,customer_ID,B_18_mean,B_18_min,B_18_max,B_2_mean,B_2_min,B_2_max,B_3_mean,B_3_min,B_3_max,...,D_120_count_9,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.590820,0.585449,0.593262,0.898438,0.810547,1.009766,0.009735,0.002348,0.020004,...,0,0,0,0,0,0,1,0,1,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.004883,1.000977,1.008789,1.003906,1.000977,1.008789,0.006161,0.001596,0.011635,...,0,0,0,0,1,1,0,0,1,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748535,0.584473,1.009766,0.901367,0.810059,1.009766,0.008553,0.000155,0.015945,...,0,0,0,0,1,1,0,0,1,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.145020,0.136230,0.165649,0.090881,0.032684,0.205688,0.666992,0.389160,0.960938,...,0,0,0,0,1,1,0,0,0,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.210327,0.205566,0.213257,0.041931,0.020172,0.106750,0.616699,0.350586,0.889648,...,0,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.846680,0.647949,1.006836,0.815430,0.811035,0.819336,0.024124,0.020874,0.028900,...,0,0,0,0,1,1,0,0,1,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.436768,0.186279,1.005859,0.411621,0.082520,1.009766,0.037567,0.007572,0.070496,...,0,0,0,0,1,1,0,0,0,1
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.531738,0.151123,0.592285,0.902344,0.171143,1.009766,0.028580,0.002672,0.079468,...,0,0,0,0,1,1,0,0,0,1
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.018127,0.009804,0.087463,0.134888,0.103333,0.192871,0.119263,0.099915,0.132202,...,0,0,0,0,1,0,1,0,0,1


In [22]:
final_agg.to_csv('D:\\AMEX\\test_del_all to score.csv',index=False)

In [3]:
selected_2=pd.read_csv("D:\\AMEX\\selected_2_del_all.csv")
cols_to_read_2=list(selected_2.selected_2)
cols_to_read_2.remove('target')
cols_to_read_2

['customer_ID',
 'D_54_max',
 'D_54_min',
 'D_112_max',
 'R_27_max',
 'D_44_last obs',
 'R_3_last obs',
 'R_27_min',
 'B_8_min',
 'R_27_mean',
 'R_16_max',
 'D_113_max',
 'D_103_max',
 'D_128_max',
 'S_25_max',
 'D_54_mean',
 'D_54_last3_avg',
 'S_25_min',
 'D_112_mean',
 'D_54_last obs',
 'D_122_max',
 'D_51_max',
 'R_16_last3_avg',
 'P_2_last obs',
 'D_143_max',
 'D_80_last obs',
 'D_44_max',
 'R_3_min',
 'D_39_last obs',
 'B_7_last obs',
 'R_1_last3_avg',
 'D_65_last obs',
 'D_82_last obs',
 'B_3_last obs',
 'D_70_max',
 'B_9_last obs',
 'D_82_min',
 'D_113_min',
 'B_33_last obs',
 'D_70_last obs',
 'R_2_last obs',
 'S_3_max',
 'D_41_last obs',
 'D_42_mean',
 'D_122_min',
 'S_3_last obs',
 'S_3_mean']